In [2]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

# 날짜 기준으로 데이터 분할
train_data = data[data['날짜'] <= pd.Timestamp('2019-12-31')]
test_data = data[data['날짜'] >= pd.Timestamp('2020-01-01')]
train_data.shape, test_data.shape

# 데이터 불러오기
data = pd.read_excel(r'C:\apps\ml\project1\datasets\all_features_testing_2m.xlsx')

# 날짜 데이터를 datetime 형식으로 변환
data['날짜'] = pd.to_datetime(data['날짜'])

# 훈련 및 테스트 피처로 분리
# train set
X_train = train_data.drop(['날짜', 'label'], axis=1)

# 독립 변수에 상수 항 추가
X_train = sm.add_constant(X_train)

# VIF 계산
vif = pd.DataFrame()
vif["Features"] = X_train.columns
vif["VIF Factor"] = [variance_inflation_factor(X_train.values, i) for i in range(X_train.shape[1])]

# VIF 값으로 내림차순 정렬 후 인덱스 리셋
vif_sorted = vif.sort_values(by='VIF Factor', ascending=False).reset_index(drop=True)

vif_sorted

,Features,VIF Factor
0,수출입물가비,2.772698
1,미국생산자물가지수,2.342675
2,const,1.974059
3,WTI유가,1.827070
4,전업생산지수,1.706346
5,설비투자지수,1.571759
6,미국취업자수,1.480528
7,실업급여지급건수,1.475499
8,소비자물가지수,1.445145
9,소매판매액지수,1.441511


- 미국비농업고용자수에 비해 미국PMI가 VIF값이 낮아 다중공선성이 적으므로 채택
- 전체실업률은 청년실업률(15~29세)에 비해 VIF값이 조금 더 높았지만, 대표성은 전체실업률에 있다고 판단하여 채택
* 노란 실선은 최종피처로 선정되지 않은 7개 피처 

- 일반적으로 VIF가 10 이상이면 다중공선성이 있을 가능성이 높다고 간주하는데 위의 결과를 보면 모든 피처의 VIF Factor가 10보다 훨씬 작으므로 다중공선성의 문제는 발생하지 않음을 확인 가능. 

- 이는 모든 피처가 종속 변수에 대해 각각의 정보를 가지고 있고, 다른 피처들과 상관관계가 높지 않음을 의미하며, 결론적으로 이 모델의 독립 변수들은 다중공선성에 의한 영향을 크게 받지 않고 신뢰할 수 있음

< 프로젝트 1_ 포파인더스 피드백>
= P23) 상관계수가 0.6이상이라고 높은 게 아님. VIF를 확인해봐라 0.7이 나올 것같음
-> VIF를 같이 보여줘라. (2 정도 나올 것같다.)

= P28) 라벨 분포를 다른 거랑 비교하기. 살제보다 수축국면이 많이 잡힌 결과임.
-> 실제 통계청은 6:4(수축)임. 
-> 고수익이 아니라, 위험 관리가 목적이니, 더 보수적 관점에서 수축이 더 많이 잡혔다.

=P34) L1, L2 규제를 줄때는 알파값을 넣어라.

=P38) 코로나 시기를 왜 넣었냐, 넣은 의도를 발표할 떄 보여주기. ★
-> 왜 이렇게 잡았는지 이유 넣어주기.

= P35) 캣부스가 성능이 좋았다라고 끝내지말고 왜 캣부스트가 높았는지 설명해주기
-> 주가 데이터에서 잘 맞음. 선행연구 찾아보고 원래 금융 데이터에 잘맞음.

= P43) MDD 값이 매우 우수함. 포트폴리오의 장점임. 손해를 볼때 효용이 더 많이 깎이니까 그 관점에서 아주 좋게 나온거임

= 자산배분에서 중요하게 봐야할 지표이기때문에 마이너스 효용이 더 크기때문에 잘 봐야함.

= 워크플로우에 소르피노 추가하기

=P47) 코로나때 결과 // 샤프와 소르티노의 계산식을 보고. 왜 소르티노가 떨어졌는지. 분자값보다 분모값이 커서그럼, 코로나 기간에 분모값이 매우 컸던 결과를 넣어주기.